In [406]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [456]:
!pip install xgboost catboost


   ---------------------------------------- 0.0/102.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/102.4 MB ? eta -:--:--
   ---------------------------------------- 0.8/102.4 MB 2.8 MB/s eta 0:00:37
    --------------------------------------- 1.3/102.4 MB 2.8 MB/s eta 0:00:37
    --------------------------------------- 1.8/102.4 MB 2.5 MB/s eta 0:00:40
    --------------------------------------- 2.1/102.4 MB 2.5 MB/s eta 0:00:41
   - -------------------------------------- 2.6/102.4 MB 2.3 MB/s eta 0:00:43
   - -------------------------------------- 3.1/102.4 MB 2.3 MB/s eta 0:00:44
   - -------------------------------------- 3.4/102.4 MB 2.2 MB/s eta 0:00:45
   - -------------------------------------- 3.9/102.4 MB 2.2 MB/s eta 0:00:45
   - -------------------------------------- 4.5/102.4 MB 2.2 MB/s eta 0:00:45
   - -------------------------------------- 5.0/102.4 MB 2.3 MB/s eta 0:00:43
   -- ------------------------------------- 5.8/102.4 MB 2.3 MB/s eta 0:00:42


In [407]:
import warnings
warnings.filterwarnings('ignore')

In [552]:
data = pd.read_csv('D:/downloads/Train_Data.csv')

In [553]:
data

,SEQN,RIAGENDR,PAQ605,BMXBMI,LBXGLU,DIQ010,LBXGLT,LBXIN,age_group
0,73564.0,2.0,2.0,35.7,110.0,2.0,150.0,14.91,Adult
1,73568.0,2.0,2.0,20.3,89.0,2.0,80.0,3.85,Adult
2,73576.0,1.0,2.0,23.2,89.0,2.0,68.0,6.14,Adult
3,73577.0,1.0,2.0,28.9,104.0,NaN,84.0,16.15,Adult
4,73580.0,2.0,1.0,35.9,103.0,2.0,81.0,10.92,Adult
...,...,...,...,...,...,...,...,...,...
1961,83711.0,2.0,2.0,33.5,100.0,2.0,73.0,6.53,Adult
1962,83712.0,1.0,2.0,30.0,93.0,2.0,208.0,13.02,Adult
1963,83713.0,1.0,2.0,23.7,103.0,2.0,124.0,21.41,Adult
1964,83718.0,2.0,2.0,27.4,90.0,2.0,108.0,4.99,Adult


In [599]:
df = data.copy()
df.head(20)

,SEQN,RIAGENDR,PAQ605,BMXBMI,LBXGLU,DIQ010,LBXGLT,LBXIN,age_group
0,73564.0,2.0,2.0,35.7,110.0,2.0,150.0,14.91,Adult
1,73568.0,2.0,2.0,20.3,89.0,2.0,80.0,3.85,Adult
2,73576.0,1.0,2.0,23.2,89.0,2.0,68.0,6.14,Adult
3,73577.0,1.0,2.0,28.9,104.0,NaN,84.0,16.15,Adult
4,73580.0,2.0,1.0,35.9,103.0,2.0,81.0,10.92,Adult
5,73581.0,1.0,2.0,23.6,110.0,2.0,100.0,6.08,Adult
6,73587.0,1.0,2.0,38.7,94.0,2.0,202.0,21.11,Adult
7,73596.0,2.0,2.0,38.3,107.0,2.0,164.0,20.93,Adult
8,73607.0,1.0,2.0,38.9,89.0,2.0,113.0,17.47,Senior
9,73610.0,1.0,1.0,28.9,90.0,2.0,95.0,3.24,Adult


In [555]:
df['age_group'] = df['age_group'].fillna('Senior')

In [556]:
df['RIAGENDR'].value_counts()

RIAGENDR
2.0    994
1.0    954
Name: count, dtype: int64

In [557]:
df['PAQ605'].value_counts()

PAQ605
2.0    1606
1.0     346
7.0       1
Name: count, dtype: int64

In [558]:
df['DIQ010'].value_counts()

DIQ010
2.0    1879
3.0      50
1.0      19
Name: count, dtype: int64

In [559]:
def func(series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    series_iqr = series[(series >= lower_bound) & (series <= upper_bound)]
    return series.fillna(series_iqr.median())

for i in df['RIAGENDR'].unique():
    for j in df['PAQ605'].unique():
        subset = df[(df['RIAGENDR'] == i) & (df['PAQ605'] == j)]
        for col in ['BMXBMI', 'LBXGLU', 'LBXIN', 'LBXGLT']:
                if not subset.empty:
                    df.loc[subset.index, col] = func(df.loc[subset.index, col])

In [560]:
df['age_group'] = df['age_group'].map({'Adult': 0, 'Senior': 1})
df = df.drop(columns=['SEQN'],axis=1)

In [561]:
df['DIQ010'] = df['DIQ010'].fillna(2.0)

In [562]:
df['PAQ605'] = df['PAQ605'].fillna(2.0)

In [563]:
df['RIAGENDR'] = df['RIAGENDR'].fillna(1.0)

In [564]:
df.isna().sum()

RIAGENDR     0
PAQ605       0
BMXBMI       0
LBXGLU       0
DIQ010       0
LBXGLT       0
LBXIN        0
age_group    0
dtype: int64

In [588]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV

X = df.drop(columns=['age_group'])
y = df['age_group']
class_counts = y.value_counts()
scale_weight = class_counts[0] / class_counts[1]

def evaluate_model(X, y, model, k=10):
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
    f1_scores = []

    for train_idx, val_idx in skf.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)

        sm = SMOTE(sampling_strategy='not majority', random_state=42)
        X_resampled, y_resampled = sm.fit_resample(X_train_scaled, y_train)
        model.fit(X_resampled, y_resampled)
        y_pred = model.predict(X_val_scaled)

        score = f1_score(y_val, y_pred)
        f1_scores.append(score)
    return np.mean(f1_scores)

model = XGBClassifier(use_label_encoder=False, eval_metric='logloss',scale_pos_weight = scale_weight,learning_rate=0.05,max_depth=4,n_estimators=300,subsample=0.8,colsample_bytree=0.8,random_state=42)
#model = SVC(kernel='rbf', class_weight='balanced')
#model = CatBoostClassifier(verbose=0)
avg_f1 = evaluate_model(X, y, model)
#avg_f1 = evaluate_model(X_scaled, y.values, model)

print(f"Average F1 Score: {avg_f1:.4f}")

Average F1 Score: 0.3673


In [589]:
test = pd.read_csv('D:/downloads/Test_Data.csv')
test

,SEQN,RIAGENDR,PAQ605,BMXBMI,LBXGLU,DIQ010,LBXGLT,LBXIN
0,77017.0,1.0,1.0,32.2,96.0,2.0,135.0,15.11
1,75580.0,2.0,2.0,26.3,100.0,2.0,141.0,15.26
2,73820.0,1.0,2.0,28.6,107.0,2.0,136.0,8.82
3,80489.0,2.0,1.0,22.1,93.0,2.0,111.0,12.13
4,82047.0,1.0,1.0,24.7,91.0,2.0,105.0,3.12
...,...,...,...,...,...,...,...,...
307,74150.0,2.0,2.0,21.9,82.0,2.0,82.0,2.54
308,82550.0,2.0,1.0,33.3,95.0,2.0,77.0,6.36
309,77835.0,2.0,2.0,41.5,91.0,2.0,149.0,15.52
310,79281.0,2.0,2.0,22.5,82.0,2.0,93.0,1.39


In [590]:
test_new = test.drop(['SEQN'], axis=1)

In [591]:
#test_new = test_new.fillna(test_new.median(numeric_only=True))

In [592]:
X_test = scaler.transform(test_new)

In [593]:
g = df.drop(['age_group'], axis=1)
h = df['age_group']

In [594]:
model.fit(g,h)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None,
              num_parallel_tree=None, ...)

In [595]:
y_pred = model.predict(test_new)

In [596]:
df_pred = pd.DataFrame(y_pred,columns=['age_group'])

In [597]:
df_pred.to_csv('out18.csv', index= False)